In [2]:
import os 
import numpy as np

In [4]:
from openfermion import MolecularData
from openfermion.ops import QubitOperator

In [9]:
mol_to_geometry = {
    'H2': [['H', [0, 0, 0]],
           ['H', [0, 0, 0.734]]],
    'LiH': [['Li', [0, 0, 0]],
            ['H', [0, 0, 1.548]]],
    'NH3': [['N', [0, 0, 0.149]],
            ['H', [0, 0.947, -0.348]],
            ['H', [0.821, -0.474, -0.348]],
            ['H', [-0.821, -0.474, -0.348]]],
    'C2': [['C', [0, 0, 0]],
           ['C', [0, 0, 1.26]]],
    'N2': [['N', [0, 0, 0]],
           ['N', [0, 0, 1.19]]],
    'H2O': [['H', [0, 0.769, -0.546]],
            ['H', [0, -0.769, -0.546]],
            ['O', [0, 0, 0.137]]],
    'hydrazine': [['N', (0, 0, 1.26135)],
                  ['N', (0, 0, -1.26135)],
                  ['H', (0, 1.7439, 2.33889)],
                  ['H', (0, -1.7439, 2.33889)],
                  ['H', (0, 1.7439, -2.33889)],
                  ['H', (0, -1.7439, -2.33889)]]
}

mol_to_spin = {
    'H2': 1,
    'LiH': 1,
    'NH3': 1,
    'N2': 1,
    'O2': 3,
    'H2O': 1,
}

mol_to_electron_num = {
    'H2': 0,
    'LiH': 0,
    'NH3': 0,
    'N2': 0,
    'O2': 0,
    'H2O': 0,
}

DEFAULT_MOLECULE_ROOT = './molecules'
DEFAULT_MOLECULE_FILENAME = 'molecular_data.hdf5' 

## Fetching the molecule and its main properties

In [10]:
mol_name = 'H2'
molecule_filename = os.path.join(molecule_dir, DEFAULT_MOLECULE_FILENAME)
mol = MolecularData(mol_to_geometry[mol_name],
                    'sto-3g',
                    mol_to_spin[mol_name],
                    mol_to_electron_num[mol_name],
                    filename=mol_filename)

NameError: name 'mol_filename' is not defined

In [16]:
mol, mol_dir, of_ham, logger = load_molecule(molecule_name='H2O',
                                             create_hamiltonian=True)
print(f'Hartree-Fock energy: {mol.hf_energy}')
print(f'FCI energy: {mol.fci_energy}')
print(f'FCI energy up to chemical accuracy: {mol.fci_energy + CHEMICAL_ACCURACY}')

For H2O molecule .hdf5 was found, loading the molecule...
The molecule was successfully loaded!
Hartree-Fock energy: -74.96254759321404
FCI energy: -75.02329149983605
FCI energy up to chemical accuracy: -75.02169149983605


In [17]:
visible_num = mol.n_orbitals * 2
electron_num = mol.n_electrons
spin = (mol.multiplicity - 1) // 2

hf_idx = (2 ** electron_num - 1) << (visible_num - electron_num)

cpu_device = pt.device('cpu')
gpu_device = pt.device('cuda:0')
work_device = gpu_device if pt.cuda.is_available() else cpu_device
work_dtype = BASE_COMPLEX_TYPE

### Obtaining Hamiltonians

In [28]:
ham_mps_num = 1
ham_mpses = []
ham_mpses_dir = os.path.join(mol_dir,
                             f'{ham_mps_num}_ham_mpses_{work_dtype}')
if os.path.exists(ham_mpses_dir):
    print(f'Hamiltonians exist on the disk, we load them')
    for ham_idx in range(ham_mps_num):
        tensor_list = []
        for idx in range(visible_num):
            tensor_list.append(pt.from_numpy(np.load(os.path.join(ham_mpses_dir,
                                                                  f'ham_mps_{ham_idx}_tensor_{idx}.npy'))))
        ham_mpses.append(MPS.from_tensor_list(tensor_list))
        print(ham_mpses[-1])
else:
    print(f'Hamiltonians do not exist on the disk, you have to create them manually!')

Hamiltonians exist on the disk, we load them
MPS None:
	visible_num = 14
	phys_dims = [4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
	bond_dims = [4, 16, 33, 54, 69, 60, 77, 92, 71, 46, 33, 16, 4]
	ext_bond_dims = [1, 4, 16, 33, 54, 69, 60, 77, 92, 71, 46, 33, 16, 4, 1]
	orth_idx = None



### Creating Hamiltonians if they don't exist

In [19]:
def pauli_strings_to_terms(pauli_strings):
    terms = []
    for pauli_string in pauli_strings:
        term = ()
        for qubit_idx, pauli_char in enumerate(pauli_string[::-1]):
            if pauli_char != 'I':
                term += (tuple((qubit_idx, pauli_char)),)
        terms.append(term)
        
    return terms

In [20]:
def pauli_strings_to_of_ham(pauli_strings, parent_of_ham):
    weighted_terms = {}
    terms = pauli_strings_to_terms(pauli_strings)
    weighted_terms.update({
        term: parent_of_ham.terms[term] for term in terms
    })
    of_ham = 0.0 * QubitOperator(' ')
    of_ham.compress()
    of_ham.terms = weighted_terms
    
    return ham   

In [21]:
def node_list_to_tree_ham_mps(node_list, parent_of_ham, dtype, verbose=False):
    weighted_terms = {}
    for node in node_list:
        pauli_strings = node.get_pauli_strings()
        terms = pauli_strings_to_terms(pauli_strings)
        weighted_terms.update({
            term: parent_of_ham.terms[term] for term in terms
        })
    of_ham = 0.0 * QubitOperator(' ')
    of_ham.compress()
    of_ham.terms = weighted_terms
    tree_ham = TreeHamiltonian(qubit_num=visible_num,
                               of_hamiltonian=of_ham,
                               dtype=dtype)
    tree_ham.compress(verbose=verbose)
    
    tree_ham_mps = tree_ham.to_mps()
    tree_ham_mps.trim_bond_dims()
    
    return tree_ham_mps

In [22]:
tree_ham = TreeHamiltonian(qubit_num=visible_num,
                           of_hamiltonian=of_ham,
                           dtype=work_dtype)
tree_ham.compress(verbose=False)
tree_ham_mps = tree_ham.to_mps()
tree_ham_mps.trim_bond_dims()
print(tree_ham_mps)
print(max(tree_ham_mps.bond_dims))

Parsing of OF QubitOperator started...
Finished!
MPS None:
	visible_num = 14
	phys_dims = [4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
	bond_dims = [4, 16, 33, 54, 69, 60, 77, 92, 71, 46, 33, 16, 4]
	ext_bond_dims = [1, 4, 16, 33, 54, 69, 60, 77, 92, 71, 46, 33, 16, 4, 1]
	orth_idx = 12

92


Choose level of tree at which we want to perform splitting (presumably the closer to the middle, the more fine-grained are the partitions)

In [23]:
split_layer_idx = 5
term_nums = pt.zeros((len(tree_ham.tree[split_layer_idx]), ), dtype=pt.int)
for node_idx, node in enumerate(tree_ham.tree[split_layer_idx]):
    print(f'Node #{node_idx}')
    cur_term_num = len(node.get_pauli_strings())
    print(f'Number of terms: {cur_term_num}\n')
    term_nums[node_idx] = cur_term_num
print(f'Total number of terms: {term_nums.sum()}')

Node #0
Number of terms: 214

Node #1
Number of terms: 54

Node #2
Number of terms: 54

Node #3
Number of terms: 54

Node #4
Number of terms: 54

Node #5
Number of terms: 42

Node #6
Number of terms: 42

Node #7
Number of terms: 42

Node #8
Number of terms: 42

Node #9
Number of terms: 22

Node #10
Number of terms: 22

Node #11
Number of terms: 22

Node #12
Number of terms: 22

Node #13
Number of terms: 22

Node #14
Number of terms: 20

Node #15
Number of terms: 20

Node #16
Number of terms: 10

Node #17
Number of terms: 10

Node #18
Number of terms: 10

Node #19
Number of terms: 10

Node #20
Number of terms: 10

Node #21
Number of terms: 10

Node #22
Number of terms: 10

Node #23
Number of terms: 10

Node #24
Number of terms: 3

Node #25
Number of terms: 12

Node #26
Number of terms: 12

Node #27
Number of terms: 3

Node #28
Number of terms: 6

Node #29
Number of terms: 6

Node #30
Number of terms: 6

Node #31
Number of terms: 6

Node #32
Number of terms: 6

Node #33
Number of terms: 

In [24]:
ham_mpses_num = 1
terms_per_mps = term_nums.sum() // ham_mpses_num
print(f'Terms per MPS: {terms_per_mps}')

Terms per MPS: 1086


/tmp/ipykernel_24222/1519806481.py:2: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  terms_per_mps = term_nums.sum() // ham_mpses_num


In [25]:
partitions = [[] for part_idx in range(ham_mpses_num)]
part_term_nums = pt.zeros((ham_mpses_num, ), dtype=pt.int)
cur_part_idx = 0
for node_idx, node in enumerate(tree_ham.tree[split_layer_idx]):
    cur_term_num = len(node.get_pauli_strings())
    partitions[cur_part_idx].append(node)
    part_term_nums[cur_part_idx] += cur_term_num
    if (part_term_nums[cur_part_idx] + cur_term_num > terms_per_mps) and (cur_part_idx != ham_mpses_num - 1):
        cur_part_idx += 1

# HERE YOU CAN REVAMP PARTITIONS

In [26]:
tree_ham_mpses = []
for partition in partitions:
    tree_ham_mpses.append(node_list_to_tree_ham_mps(partition, of_ham, dtype=work_dtype))
    print(tree_ham_mpses[-1])

Parsing of OF QubitOperator started...
Finished!
MPS None:
	visible_num = 14
	phys_dims = [4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
	bond_dims = [4, 16, 33, 54, 69, 60, 77, 92, 71, 46, 33, 16, 4]
	ext_bond_dims = [1, 4, 16, 33, 54, 69, 60, 77, 92, 71, 46, 33, 16, 4, 1]
	orth_idx = 12



In [27]:
ham_mpses_dir = os.path.join(mol_dir,
                             f'{ham_mps_num}_ham_mpses_{work_dtype}')

if not os.path.exists(ham_mpses_dir):
    os.makedirs(ham_mpses_dir)
    
ham_mpses = tree_ham_mpses
assert len(ham_mpses) == ham_mpses_num
for ham_idx, ham_mps in enumerate(ham_mpses):
    for idx in range(visible_num):
        np.save(os.path.join(ham_mpses_dir,
                             f'ham_mps_{ham_idx}_tensor_{idx}.npy'),
                ham_mps.tensors[idx])